# Object Tracking

## Single object tracking

In [1]:
import cv2
import imutils

TrDict = {'csrt': cv2.legacy.TrackerCSRT_create,
         'kcf' : cv2.legacy.TrackerKCF_create,
         'boosting' : cv2.legacy.TrackerBoosting_create,
         'mil': cv2.legacy.TrackerMIL_create,
         'tld': cv2.legacy.TrackerTLD_create,
         'medianflow': cv2.legacy.TrackerMedianFlow_create,
         'mosse':cv2.legacy.TrackerMOSSE_create}

tracker = TrDict['csrt']()
#tracker = cv2.TrackerCSRT_create()

v = cv2.VideoCapture(r'video1.mp4') 


ret, frame = v.read()
#frame = imutils.resize(frame,width=600)
cv2.imshow('Frame',frame)
bb = cv2.selectROI('Frame',frame)
tracker.init(frame,bb)


while True:
    ret, frame = v.read()
    if not ret:
        break
    #frame = imutils.resize(frame,width=600)
    (success,box) = tracker.update(frame)
    if success:
        (x,y,w,h) = [int(a) for a in box]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(100,255,0),2)
    cv2.imshow('Frame',frame)
    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        break
v.release()
cv2.destroyAllWindows()

## Multi Object Tracking
#### Change the 'k' variable to change the number of objects to be tracked

In [2]:
import cv2
import numpy as np
import os

TrDict = {'csrt': cv2.legacy.TrackerCSRT_create,
          'kcf': cv2.legacy.TrackerKCF_create,
          'boosting': cv2.legacy.TrackerBoosting_create,
          'mil': cv2.legacy.TrackerMIL_create,
          'tld': cv2.legacy.TrackerTLD_create,
          'medianflow': cv2.legacy.TrackerMedianFlow_create,
          'mosse': cv2.legacy.TrackerMOSSE_create}

trackers = cv2.legacy.MultiTracker_create()

v = cv2.VideoCapture(r'video2.mp4')

ret, frame = v.read()
if not ret:
    print("Failed to read video file.")
    exit()

k = 2
for i in range(k):
    cv2.imshow('Frame', frame)
    bbi = cv2.selectROI('Frame', frame, fromCenter=False, showCrosshair=True)
    if bbi == (0, 0, 0, 0):
        break
    tracker_i = TrDict['csrt']()
    trackers.add(tracker_i, frame, bbi)

baseDir = r'TrackingResults'
if not os.path.exists(baseDir):
    os.makedirs(baseDir)

frameNumber = 2
while True:
    ret, frame = v.read()
    if not ret:
        break

    success, boxes = trackers.update(frame)
    
    np.savetxt(f'{baseDir}/frame_{frameNumber}.txt', boxes, fmt='%f')
    
    for box in boxes:
        (x, y, w, h) = [int(a) for a in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv2.imshow('Frame', frame)
    frameNumber += 1

    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        break

v.release()
cv2.destroyAllWindows()